In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets bitsandbytes einops wandb
!pip install -q datasets
!pip install git+https://github.com/huggingface/accelerate
!pip install --upgrade transformers
!pip install transformers accelerate peft bitsandbytes datasets einops tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install csv

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [ ]:
import pandas as pd
import csv

In [ ]:
from huggingface_hub import login
login()

Taking dataset


In [ ]:
dp= pd.read_csv('/content/data.csv')

In [ ]:
dp

,Questions,Answers,text
0,"Problem in- PC, Problem type is- Does not star...",currupt profile reset,you are given details on an issue faced by our...
1,"Problem in- PRINTER, Problem type is- Other - ...",Changed setting,you are given details on an issue faced by our...
2,"Problem in- PC, Problem type is- Does not star...",config properly,you are given details on an issue faced by our...
3,"Problem in- PRINTER, Problem type is- Other - ...",Resolved issue,you are given details on an issue faced by our...
4,"Problem in- PC, Problem type is- Other - Pleas...",Changed setting,you are given details on an issue faced by our...
...,...,...,...
40598,"Problem in- OTHER, Problem type is- Other - Pl...",user not avliable calling again2,you are given details on an issue faced by our...
40599,"Problem in- PC, Problem type is- Virus, Proble...",ok problem resolved.,you are given details on an issue faced by our...
40600,"Problem in- PC, Problem type is- Working slow,...",user not known abt the compl. user said leave it,you are given details on an issue faced by our...
40601,"Problem in- SCANNER, Problem type is- Other - ...",scanner settings done,you are given details on an issue faced by our...


# New section

# New section

In [ ]:
info="there are mentioned the details with respect to problems faced by the customer, Write a response that appropriately solves the issue faced by the customer."

In [ ]:
input_file="/content/data.csv"
concatenated_list = []

with open(input_file, 'r', newline='', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        concatenated_text = (
            info
            + " "
            + "###Human "
            + row["Questions"]
            + " ###Assistent "
            + row["Answers"]
        )
        concatenated_list.append(concatenated_text)

In [ ]:
def format_input(example: dict) ->str:
  example['formated_text']= f'''###Human: Enter the query: {example["Questions"]}
  ### Assistant: {example["Answers"]}'''
  return example

In [ ]:
dp= dp.map(concatenated_list)

AttributeError: ignored

In [ ]:
concatenated_list[2]

'there are mentioned the details with respect to problems faced by the customer, Write a response that appropriately solves the issue faced by the customer. ###Human Problem in- PC, Problem type is- Does not start, Problem description is- new pc configration and domain add, Name of Equipment- PC GEN, Service Company is- WIPRO ###Assistent config properly'

importing features from dataset libraries


In [ ]:
from datasets import DatasetDict, Dataset
dset = {
    "text": concatenated_list,
}
dataset = Dataset.from_dict(dset)

In [ ]:
dp= dp.map(dataset)

AttributeError: ignored

# load the Falcon 7B model

Here we will load the Falcon 7B model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import prepare_model_for_kbit_training
model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map='auto'
)
model.config.use_cache = False
model.gradient_checkpointing_enable()

model= prepare_model_for_kbit_training(model)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loading tokens

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Then we have to apply some preprocessing to the model to prepare it for training. For that use the prepare_model_for_kbit_training method from PEFT.


In [ ]:
from peft import LoraConfig,  get_peft_model, TaskType

lora_alpha = 32
lora_dropout = 0.05
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "query_key_value"
        # "dense",
        # "dense_h_to_4h",
        # "dense_4h_to_h",
    ]
)

In [ ]:
from peft import get_peft_model

model= get_peft_model(model, peft_config)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
num_train_epochs=1,
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 150
warmup_ratio = 0.05
lr_scheduler_type = "cosine"

training_arguments = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

In [ ]:
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 40603
})

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments
)

Map:   0%|          | 0/40603 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

Training the Model

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,4.246500
20,3.505200
30,1.937800
40,1.001000
50,0.680900
60,1.208200
70,0.834800
80,0.657500
90,0.500400
100,0.392300


TrainOutput(global_step=150, training_loss=1.2221491114298502, metrics={'train_runtime': 1310.185, 'train_samples_per_second': 1.832, 'train_steps_per_second': 0.114, 'total_flos': 7729663742819328.0, 'train_loss': 1.2221491114298502, 'epoch': 0.06})

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
trained_model_dir = './trained_model'
model.save_pretrained(trained_model_dir)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [ ]:
device_map = {
    "transformer.word_embeddings": 0,
    "transformer.word_embeddings_layernorm": 0,
    "lm_head": "cpu",
    "transformer.h": 0,
    "transformer.ln_f": 0,
}

In [ ]:
model_8bit = AutoModelForCausalLM.from_pretrained(
    "ybelkada/falcon-7b-sharded-bf16",
    device_map=device_map,
    quantization_config=quantization_config,
)

Loading ybelkada/falcon-7b-sharded-bf16 requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/ybelkada/falcon-7b-sharded-bf16. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: ignored

In [ ]:
from peft import PeftConfig, PeftModel
config = PeftConfig.from_pretrained(trained_model_dir)

trained_model = AutoModelForCausalLM.from_pretrained(
  config.base_model_name_or_path,
  return_dict=True,
  quantization_config=bnb_config,
  trust_remote_code=True,
  device_map='auto'
)
trained_model = PeftModel.from_pretrained(trained_model, trained_model_dir)

trained_model_tokenizer = AutoTokenizer.from_pretrained(config.base_model_nam_or_path, trust_remote_code=True)
trained_model_tokenizer.pad_token = trained_model_tokenizer.eos_token


ValueError: ignored

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Parameter

In [ ]:
generation_config= model.generation_config
generation_config.max_new_token= 1024
generation_config.temperature= 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequence = 1
generation_config.pad_token_id = tokenizer.pad_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_token": 1024,
  "num_return_sequence": 1,
  "pad_token_id": 11,
  "temperature": 0.7,
  "top_p": 0.7
}

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

TypeError: ignored

In [ ]:
device= 'cuda:0'

In [ ]:
query='''A new large language model was developed using a sizable taxt and code training dataset, which is helping in providing with solution. '''

In [ ]:
prompt= f'''###Human: my password expired, please help me?: {query}
###Assistent: '''

In [ ]:
encodings= tokenizer(prompt, return_tensors= 'pt').to(device)

In [ ]:
encodings

{'input_ids': tensor([[19468, 23431,    37,   491,  7363, 27727,    23,  2551,   733,   454,
         16232,   317,   627,  1902,  3599,  2308,   398,  4027,  1241,   241,
         49242, 11516,   689,   273,  2928,  2555, 20512,    23,   585,   304,
          4851,   272,  3839,   335,  3377,    25, 13889, 19468,  6939,  9358,
            37,   204]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
with torch.inference_mode():
  outputs = model.generate(
      input_ids=encodings.input_ids,
      attention_mask=encodings.attention_mask,
      generation_config=generation_config,
      max_new_tokens=100
  )

In [ ]:
outputs

tensor([[19468, 23431,    37,   491,  7363, 27727,    23,  2551,   733,   454,
         16232,   317,   627,  1902,  3599,  2308,   398,  4027,  1241,   241,
         49242, 11516,   689,   273,  2928,  2555, 20512,    23,   585,   304,
          4851,   272,  3839,   335,  3377,    25, 13889, 19468,  6939,  9358,
            37,   204,    37,   317,   627,  1902,  3599,  2308,   398,  4027,
          1241,   241, 49242, 11516,   689,   273,  2928,  2555, 20512,    23,
           585,   304,  4851,   272,  3839,   335,  3377,    25, 13889, 19468,
         23431,    37,   204,    37,   317,   627,  1902,  3599,  2308,   398,
          4027,  1241,   241, 49242, 11516,   689,   273,  2928,  2555, 20512,
            23,   585,   304,  4851,   272,  3839,   335,  3377,    25, 13889,
         19468,  6939,  9358,    37,   204,    37,   317,   627,  1902,  3599,
          2308,   398,  4027,  1241,   241, 49242, 11516,   689,   273,  2928,
          2555, 20512,    23,   585,   304,  4851,  

In [ ]:
outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)

outputs

'###Human: my password expired, please help me?: A new large language model was developed using a sizable taxt and code training dataset, which is helping in providing with solution.  \n###Assistent: : A new large language model was developed using a sizable taxt and code training dataset, which is helping in providing with solution.  \n###Human: : A new large language model was developed using a sizable taxt and code training dataset, which is helping in providing with solution.  \n###Assistent: : A new large language model was developed using a sizable taxt and code training dataset, which is helping in providing with solution.  \n###Human: : A new large language model'

In [ ]:
text = "###Human VERY SLOW NOT WORKING MANYTIME PLS REPLACE WITH NEW SCANNER ###Assistent"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
for k in outputs:
  print(tokenizer.decode(k, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


ValueError: ignored